## config

In [32]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MultiLabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
import matplotlib.pyplot as plt
import re

from sklearn.feature_extraction.text import CountVectorizer

import json
import glob

import pickle

In [2]:
pd.set_option('display.float_format', '{:.4f}'.format)

## from EDA

In [3]:
all_data = []
for file in glob.glob("../data/processed/*.json"):
    print(file)
    with open(file, "r") as f:
        all_data.extend(json.load(f))

df = pd.DataFrame(all_data)
df = df.dropna(thresh=8)
df = df.reset_index(drop=True)

../data/processed\channels0_99.json
../data/processed\channels100_.json
../data/processed\channels3425_.json
../data/processed\channels6727_.json
../data/processed\channelsa10034_.json
../data/processed\channelsa13313_.json
../data/processed\channels_missing.json


In [4]:
df["created_date"] = pd.to_datetime(df["created_date"], format="ISO8601")

In [5]:
time_cols = ["created_date"]
num_cols = ["avg_duration_seconds", "avg_seconds_between_uploads"]
basic_cat_cols = ["country", "most_common_video_genre"]
multi_cat_cols = ["category", "all_video_genres"]
text_cols = ["description", "aggregated_tags", "recent_video_titles"]
drop_cols = ["channel_id", "channel_name", "defaultLanguage"]
set(time_cols + num_cols + basic_cat_cols + multi_cat_cols + text_cols + drop_cols) == set(df.columns)

True

TODO: investigate below scaling step

In [6]:
df["avg_seconds_between_uploads"] = df["avg_seconds_between_uploads"].fillna(df["avg_seconds_between_uploads"].max())

In [84]:
class DateTimeToPosix(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return (pd.to_datetime(X.iloc[:, 0]).astype(int) // 10**9).values.reshape(-1, 1)
    def get_feature_names_out(self, input_features=None):
        return ["created_date_posix"]

In [90]:
# MLB does not have fit/transform. must wrap in custom transformer, give it fit/transform, and col transformer will accept
class MultiLabelBinarizerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlb = MultiLabelBinarizer()
    
    def fit(self, X, y=None):
        self.mlb.fit(X.iloc[:, 0])
        return self
    
    def transform(self, X, y=None):
        return self.mlb.transform(X.iloc[:, 0])
    
    def get_feature_names_out(self, input_features=None):
        return self.mlb.classes_


## text embeddings experimentation

### nltk/spacy playing

In [80]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import spacy

In [10]:
nlp = spacy.load("en_core_web_md")
doc = nlp("pineapple") 
doc.vector[:10]

array([-0.63358,  0.12266,  0.47232, -0.22974, -0.26307,  0.56499,
       -0.72338,  0.16736,  0.4203 ,  0.93788], dtype=float32)

In [15]:
text = (
    "UBC is one of the well known universities in British Columbia. "
    "UBC CS teaching team is truly multicultural!! "
    "Dr. Toti completed her Ph.D. in Italy."
    "Dr. Moosvi, Dr. Kolhatkar, and Dr. Ola completed theirs in Canada."
    "Dr. Heeren and Dr. Lécuyer completed theirs in the U.S."
)
print(text.split("."))

sent_tokenized = sent_tokenize(text)
print(sent_tokenized)

['UBC is one of the well known universities in British Columbia', ' UBC CS teaching team is truly multicultural!! Dr', ' Toti completed her Ph', 'D', ' in Italy', 'Dr', ' Moosvi, Dr', ' Kolhatkar, and Dr', ' Ola completed theirs in Canada', 'Dr', ' Heeren and Dr', ' Lécuyer completed theirs in the U', 'S', '']
['UBC is one of the well known universities in British Columbia.', 'UBC CS teaching team is truly multicultural!!', 'Dr. Toti completed her Ph.D. in Italy.Dr.', 'Moosvi, Dr. Kolhatkar, and Dr. Ola completed theirs in Canada.Dr.', 'Heeren and Dr. Lécuyer completed theirs in the U.S.']


In [16]:
print("Splitting on whitespace: ", [sent.split() for sent in sent_tokenized])
print("Splitting on whitespace: ", [word_tokenize(sent) for sent in sent_tokenized])

Splitting on whitespace:  [['UBC', 'is', 'one', 'of', 'the', 'well', 'known', 'universities', 'in', 'British', 'Columbia.'], ['UBC', 'CS', 'teaching', 'team', 'is', 'truly', 'multicultural!!'], ['Dr.', 'Toti', 'completed', 'her', 'Ph.D.', 'in', 'Italy.Dr.'], ['Moosvi,', 'Dr.', 'Kolhatkar,', 'and', 'Dr.', 'Ola', 'completed', 'theirs', 'in', 'Canada.Dr.'], ['Heeren', 'and', 'Dr.', 'Lécuyer', 'completed', 'theirs', 'in', 'the', 'U.S.']]
Splitting on whitespace:  [['UBC', 'is', 'one', 'of', 'the', 'well', 'known', 'universities', 'in', 'British', 'Columbia', '.'], ['UBC', 'CS', 'teaching', 'team', 'is', 'truly', 'multicultural', '!', '!'], ['Dr.', 'Toti', 'completed', 'her', 'Ph.D.', 'in', 'Italy.Dr', '.'], ['Moosvi', ',', 'Dr.', 'Kolhatkar', ',', 'and', 'Dr.', 'Ola', 'completed', 'theirs', 'in', 'Canada.Dr', '.'], ['Heeren', 'and', 'Dr.', 'Lécuyer', 'completed', 'theirs', 'in', 'the', 'U.S', '.']]


### embedding/dataset pruning

#### list -> free text

In [44]:
df["aggregated_tags"] = df["aggregated_tags"].apply(lambda x: " ".join(x) if isinstance(x, list) else x)
df["recent_video_titles"] = df["recent_video_titles"].apply(lambda x: " ".join(x) if isinstance(x, list) else x)

In [48]:
df.head()

,channel_id,channel_name,description,country,defaultLanguage,created_date,category,aggregated_tags,most_common_video_genre,all_video_genres,avg_duration_seconds,avg_seconds_between_uploads,recent_video_titles
0,UCOmHUn--16B90oW2L6FRR3A,BLACKPINK,BLACKPINK Official YouTube Channel\n블랙핑크 공식 유튜...,KR,None,2016-06-29 03:15:23+00:00,"[Electronic music, Pop music, Music of Asia, M...",JISOO YG JENNIE You & Me LISA JENNIE 유앤미 와이지 블...,Music,[Music],211.6000,1017992.8900,BLACKPINK - WORLD TOUR [DEADLINE] IN HONG KONG...
1,UC3IZKseVpdzPSBaWxBxundA,HYBE LABELS,Welcome to the official YouTube channel of HYB...,KR,None,2008-06-04 08:23:22+00:00,"[Pop music, Music, Music of Asia]",하이브 하이브레이블즈 HYBE LABELS HYBE,Music,[Music],89.6000,131588.1100,SANTOS BRAVOS “KAWASAKI (&TEAM Remix)” Lyric V...
2,UCF1JIbMUs6uqoZEY1Haw0GQ,Shemaroo,"Welcome to ShemarooEnt, one of the finest dest...",IN,None,2007-09-01 11:44:51+00:00,"[Film, Entertainment]",salman khan movies ramcharana moves Mega Power...,Entertainment,[Entertainment],5336.1000,45200.0000,Mega Power Star Ram Charan 👑 | Zanjeer (4K Act...
3,UCYiGq8XF7YQD00x7wAd62Zg,JuegaGerman,Lento pero seguro.,CL,None,2013-05-19 00:09:13+00:00,"[Action game, Video game culture, Action-adven...",revenia juega german juego de miedo click to c...,Gaming,[Gaming],2046.9000,280466.0000,Fotos Tomadas En El Momento PERFECTO 📸 Trabajo...
4,UC4NALVCmcmL5ntpV0thoH6w,LooLoo Kids - Nursery Rhymes and Children's Songs,LooLoo Kids💖 is an educational YouTube channel...,US,en,2014-08-05 20:15:33+00:00,"[Entertainment, Music, Film]",kids videos children songs farm song nursery r...,Music,[Music],148.5000,181623.8900,Old Macdonald Had a Farm Song + Johny Johny Ye...


In [51]:
df["country"].value_counts()

country
US    4930
IN    2307
GB    1667
CA    1160
AU     958
      ... 
LA       1
OM       1
HN       1
ME       1
UM       1
Name: count, Length: 109, dtype: int64

In [50]:
english_speaking = ['US', 'GB', 'CA', 'AU', 'IE', 'NZ', 'SG', 'ZA', 'NG', 'GH', 'JM', 'VI', 'BM', 'AG']

In [52]:
df_english_country = df[df["country"].isin(english_speaking)]

In [57]:
def is_english(text):
    if not isinstance(text, str):
        return True
    # Allow ASCII + common special chars + emojis
    return bool(re.match(r'^[\x00-\x7F\u00C0-\u024F\u2000-\u206F\u2100-\u214F\u2190-\u21FF\u2600-\u27BF\U0001F300-\U0001FAFF©®™—…•""'']*$', text))

In [58]:
mask = (
    df["description"].apply(is_english) &
    df["aggregated_tags"].apply(is_english) &
    df["recent_video_titles"].apply(is_english) 
)

df_english_regex  = df[mask]

In [59]:
print(f"rows after eliminating by country: {len(df_english_country)}")
print(f"rows after  eliminating by regex: {len(df_english_regex)}")

rows after eliminating by country: 8861
rows after  eliminating by regex: 9759


In [60]:
df_english_regex.head()

,channel_id,channel_name,description,country,defaultLanguage,created_date,category,aggregated_tags,most_common_video_genre,all_video_genres,avg_duration_seconds,avg_seconds_between_uploads,recent_video_titles
6,UCX6OQ3DkcsbYNE6H8uQQuVA,MrBeast,SUBSCRIBE FOR A COOKIE!\nNew MrBeast or MrBeas...,US,en,2012-02-20 00:43:50+00:00,"[Entertainment, Lifestyle (sociology)]",,Entertainment,[Entertainment],303.8000,384399.7800,"Every Step You Take, Win $1,000 Surprising My ..."
8,UCIwFjwMjI0y7PDBVEO9-bkQ,Justin Bieber,,CA,None,2007-01-15 21:17:27+00:00,"[Soul music, Music, Pop music, Electronic musi...",sorry purpose believe anyone beauty and a beat...,Music,[Music],179.1000,307323.2200,Justin Bieber - BAD HONEY Justin Bieber - SPEE...
12,UC56gTxNs4f9xZ7Pa2i5xNzg,Sony Music India,Welcome to Sony Music India's official YouTube...,IN,None,2009-09-02 23:16:22+00:00,"[Soul music, Pop music, Music, Music of Asia, ...",panghat song best of ajay atul marathi songs j...,Music,[Music],298.1000,19199.3300,"Roohi - Audio Jukebox | Nadiyon Paar, Panghat ..."
13,UC0C-w0YjGpqDXGB8IHb662A,Ed Sheeran,Ed Sheeran x\n\nPlay - Out Now\n,None,None,2006-08-08 13:59:50+00:00,"[Pop music, Music]",ed sheeran live performance perth ed sheeran t...,Music,[Music],55.8000,554464.7800,There’s a new mayor in town Playing a throwbac...
16,UC9CoOnJkIBMdeijd9qYoT_g,Ariana Grande,eternal sunshine deluxe ♡ 📦 out now\n,None,None,2007-01-22 01:53:12+00:00,"[Music, Pop music, Electronic music]",ariana grande brighter days ahead behind the s...,Music,[Music],233.5000,1427546.2200,Ariana Grande - dandelion Ariana Grande - eter...


In [61]:
text_cols

['description', 'aggregated_tags', 'recent_video_titles']

### putting it together

In [102]:
ct = make_column_transformer(
    (make_pipeline(DateTimeToPosix(), StandardScaler()), time_cols),
    (StandardScaler(), num_cols),
    (make_pipeline(SimpleImputer(strategy="most_frequent"), OneHotEncoder(handle_unknown="ignore")), basic_cat_cols), #TODO: almost certainly a better way to impute countries
    (MultiLabelBinarizerTransformer(), ["category"]),
    (MultiLabelBinarizerTransformer(), ["all_video_genres"]),
    (CountVectorizer(stop_words="english", max_features=500), "description"), #TODO: figure out best hyperparameters
    (CountVectorizer(stop_words="english", max_features=500), "aggregated_tags"),
    (CountVectorizer(stop_words="english", max_features=500), "recent_video_titles"),
    ("drop", drop_cols)
)

if no max features actual shape is (7319, 163596), cant visualize

In [99]:
df_train, df_test = train_test_split(df_english_regex, random_state=67)

In [100]:
df_train_pp = ct.fit_transform(df_train)
df_test_pp = ct.transform(df_test)

In [101]:
col_names = ct.get_feature_names_out()

In [104]:
pd.DataFrame(df_train_pp.toarray(), columns=col_names) 

,pipeline-1__created_date_posix,standardscaler__avg_duration_seconds,standardscaler__avg_seconds_between_uploads,pipeline-2__country_AE,pipeline-2__country_AF,pipeline-2__country_AL,pipeline-2__country_AQ,pipeline-2__country_AR,pipeline-2__country_AT,pipeline-2__country_AU,...,countvectorizer-3__workout,countvectorizer-3__world,countvectorizer-3__worst,countvectorizer-3__worth,countvectorizer-3__wrong,countvectorizer-3__year,countvectorizer-3__years,countvectorizer-3__young,countvectorizer-3__youtube,countvectorizer-3__zombies
0,0.2573,-0.3629,0.1528,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,-1.2424,-0.4259,-0.2642,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.2622,0.0866,0.0629,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,1.8082,1.7933,0.0790,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.1363,0.0686,0.0781,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7314,-0.7730,0.2511,-0.2505,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7315,-1.0760,-0.3943,-0.2642,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7316,-1.0352,-0.1875,-0.1066,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,...,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7317,0.4301,-0.3887,1.3054,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [105]:
#TODO: pca, other ways to weight? one hot encoding screws up neighbours maybe?

In [106]:
nn = NearestNeighbors(n_neighbors=10, metric="cosine")
nn.fit(df_train_pp)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [112]:
distances, indices = nn.kneighbors(df_test_pp[0])
print(df_test.iloc[0]["channel_name"])
recs = df.iloc[indices[0]][["channel_name", "channel_id"]]
recs["distance"] = distances[0]
recs

Doctor D.S.


,channel_name,channel_id,distance
5996,MakeMyTripOfficial,UCkCnjfFgbcFwiY5StOdsSMA,0.4925
4617,G3AR - 'GreekGadgetGuru',UCGjJOXtT9nXYgjFFzRDdaTA,0.5051
2430,MagicalMessi,UCsObGXVUdSh-bHT7CdVXrrw,0.5130
6218,Squasha Streams,UC_d0N1gKUOfdct2KSbLFLsg,0.5277
1086,OGGY,UCNEKMkg_DG8eAyR1BNWsSvw,0.5292
5523,Railfan HS,UCY_2QxxKzBQYexlf2j051pQ,0.5349
2635,Fitnessbaby,UCa1ThvENwie_ndINHiateIA,0.5637
5172,Billy Perry,UCWhMeA7TdkaClRf2jlkcBgg,0.6055
4339,The Deshbhakt,UCmTM_hPCeckqN3cPWtYZZcg,0.6073
656,Stephen Sharer,UCDfk8ogO6QGeJAYCY0QDzKw,0.6194


In [ ]:
df_test.head()

,channel_id,channel_name,description,country,defaultLanguage,created_date,category,aggregated_tags,most_common_video_genre,all_video_genres,avg_duration_seconds,avg_seconds_between_uploads,recent_video_titles
13602,UCPJ61cXBMjpjyCcb5OhUzEg,Doctor D.S.,Follow me on other Social Networks!\nInstagram...,HR,None,2016-09-29 14:20:36+00:00,"[Hobby, Lifestyle (sociology), Vehicle]",Ultimaker HOW TO MAKE coreXY 3D printer Creali...,Autos & Vehicles,[Autos & Vehicles],666.2000,872773.1100,#einhell #tools #powertools #sanders #grind #m...
2543,UC6pGDc4bFGD1_36IKv3FnYg,Crunchyroll,"Crunchyroll brings you clips, anime openings, ...",US,None,2009-05-14 13:08:59+00:00,"[Entertainment, Film, Video game culture]",dark moon the blood altar crunchyroll podcast ...,Film & Animation,[Film & Animation],316.1000,9997.6700,The Most Beautiful Outside Block in the World ...
15408,UCd9050-P0h6mdrvgVtuRv8w,edcunhaph,"Vídeos dos mais variados esportivos do Brasil,...",BR,None,2007-12-23 01:30:00+00:00,"[Motorsport, Vehicle, Lifestyle (sociology)]",edcunhaph Compilation Fast Bikes Sound Moto GP...,Autos & Vehicles,[Autos & Vehicles],124.9000,2471940.0000,Kawasaki Ninja ZX10R - Cena de abertura do nov...
10853,UCRtsZ5Iak9wSLsQLQ3XOAeA,SciManDan,"Flat Earth debunker that uses science, reason ...",GB,None,2017-12-05 09:29:16+00:00,"[Society, Knowledge]",tiktok confidently wrong Fail sudoku funny puz...,Science & Technology,[Science & Technology],786.8000,86394.6700,All the Worst Flat Earth Arguments in One Vide...
9939,UCq3xWfTdDRjgyd7_WOK94Fg,Yung Filly,\n\nFOLLOW MY SOCIALS:\n\nTwitch: YungFilly\nT...,GB,None,2013-11-29 05:11:12+00:00,"[Film, Music, Lifestyle (sociology), Hip hop m...",filly Specs Gonzalez Blind Filly King Kenny Ke...,Entertainment,[Entertainment],670.7000,764099.7800,"THE ALPHABET GAME FT CHUNKZ, DARKEST & HARRY P..."


next steps: investigate and iterate.
- do some more predictions
- intepretability (actual numbers/values for a few rows)
- dimensionality issues (ohe, text) + PCA
- compare with predictions from previous model
- focus on a smaller subset of cols and try build up from there if predictions do not improve  
  
- then get focused dataset once improved

#### not figured out yet

In [108]:
def get_channel_recs(ch):
    distances, indices = nn.kneighbors(ch)

    recs = df.iloc[indices[0]][["channel_name", "channel_id"]]
    recs["distance"] = distances[0]
    return recs

In [109]:
for i in range(11):
    new_channel = df_test_pp.iloc[[i]]
    print(f"{i} {new_channel["channel_name"]}")

    print(get_channel_recs(new_channel))
    print("=============================")
    print("=============================")
    print("=============================")

AttributeError: 'csr_matrix' object has no attribute 'iloc'